In [225]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency, f_oneway, kruskal, mannwhitneyu
import statsmodels.formula.api as smf
import statsmodels.api as sm
from patsy import dmatrix
from scipy import stats

In [226]:
df = pd.read_csv('tyg.csv')
df.head()

,subject_id,hadm_id,stay_id,intime,age,gender,race,los,diabetes,hypertension,...,has_ercp,has_vasopressin,los_icu,los_hosp,hosp_mortality,icu_mortality,mortality_7d,mortality_28d,mortality_90d,mortality_1y
0,10004606,29242151,30213599,2159-02-20 16:10:03,64,F,WHITE,5.166100,0,1,...,0,0,5.166100,14.130556,0,0,0,0,0,0
1,10006441,24120560,37240744,2174-09-12 16:57:04,51,M,WHITE,0.776481,0,0,...,0,0,0.776481,15.834722,0,0,0,0,0,0
2,10007795,28477357,31921355,2136-04-22 18:01:13,53,F,WHITE,1.050521,0,1,...,0,0,1.050521,21.834722,0,0,0,0,0,0
3,10010888,20162667,33318955,2174-01-09 00:21:00,43,M,WHITE,9.019225,1,0,...,0,0,9.019225,17.736806,0,0,0,0,0,0
4,10012206,23961896,37675714,2127-07-04 02:08:00,45,M,WHITE,0.572477,1,1,...,0,0,0.572477,10.655556,0,0,0,0,0,0


In [227]:
df['TyG_group'] = pd.qcut(x= df['tyg'], q=4, labels= ['Q1', 'Q2', 'Q3', 'Q4'])
df.head(2)

,subject_id,hadm_id,stay_id,intime,age,gender,race,los,diabetes,hypertension,...,has_vasopressin,los_icu,los_hosp,hosp_mortality,icu_mortality,mortality_7d,mortality_28d,mortality_90d,mortality_1y,TyG_group
0,10004606,29242151,30213599,2159-02-20 16:10:03,64,F,WHITE,5.166100,0,1,...,0,5.166100,14.130556,0,0,0,0,0,0,Q3
1,10006441,24120560,37240744,2174-09-12 16:57:04,51,M,WHITE,0.776481,0,0,...,0,0.776481,15.834722,0,0,0,0,0,0,Q1


In [228]:
races = {
    'UNKNOWN': 'Others',
    'UNABLE TO OBTAIN': 'Others',
    'ASIAN - CHINESE': 'Others',
    'ASIAN - SOUTH EAST ASIAN': 'Others',
    'ASIAN - ASIAN INDIAN': 'Others',
    'ASIAN - KOREAN': 'Others',
    'ASIAN': 'Others',
    'HISPANIC OR LATINO': 'Others',
    'HISPANIC/LATINO - PUERTO RICAN': 'Others',
    'HISPANIC/LATINO - GUATEMALAN': 'Others',
    'HISPANIC/LATINO - DOMINICAN': 'Others',
    'HISPANIC/LATINO - MEXICAN': 'Others',
    'HISPANIC/LATINO - SALVADORAN': 'Others',
    'HISPANIC/LATINO - CENTRAL AMERICAN': 'Others',
    'HISPANIC/LATINO - CUBAN': 'Others',
    'HISPANIC/LATINO - COLUMBIAN': 'Others',
    'HISPANIC/LATINO - HONDURAN': 'Others',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'Others',
    'AMERICAN INDIAN/ALASKA NATIVE': 'Others',
    'MULTIPLE RACE/ETHNICITY': 'Others',
    'SOUTH AMERICAN': 'Others',
    'OTHER': 'Others',
    'PATIENT DECLINED TO ANSWER': 'Others',
    'WHITE': 'White',
    'WHITE - OTHER EUROPEAN': 'White',
    'WHITE - RUSSIAN': 'White',
    'WHITE - EASTERN EUROPEAN': 'White',
    'WHITE - BRAZILIAN': 'White',
    'PORTUGUESE': 'White',
    'BLACK/AFRICAN AMERICAN': 'Black',
    'BLACK/CARIBBEAN ISLAND': 'Black',
    'BLACK/CAPE VERDEAN': 'Black',
    'BLACK/AFRICAN': 'Black',
}
df['race'] = df['race'].map(races)

In [229]:
df.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'intime', 'age', 'gender', 'race',
       'los', 'diabetes', 'hypertension', 'wbc', 'rbc', 'rdw', 'hemoglobin',
       'platelets', 'aniongap', 'bicarbonate', 'calcium', 'chloride',
       'potassium', 'creatinine', 'glucose', 'triglycerides', 'alt', 'alp',
       'ast', 'pt', 'ptt', 'inr', 'neutrophils_max', 'neutrophils_min',
       'albumin_max', 'albumin_min', 'bun_max', 'bun_min', 'sepsis',
       'sepsis_3', 'npar', 'tyg', 'sofa', 'has_sepsis', 'aki',
       'has_chronic_kidney_disease', 'has_crrt', 'has_mv', 'has_ercp',
       'has_vasopressin', 'los_icu', 'los_hosp', 'hosp_mortality',
       'icu_mortality', 'mortality_7d', 'mortality_28d', 'mortality_90d',
       'mortality_1y', 'TyG_group'],
      dtype='object')

# Đặc điểm bệnh nhân sepsis và non-sepsis

In [230]:
# Hàm mô tả biến liên tục
def describe_continuous(var, df, results):
    group0 = df[df['sepsis'] == 0][var].dropna()
    group1 = df[df['sepsis'] == 1][var].dropna()
    overall = df[var].dropna()
    
    if len(overall) == 0:
        return results
    
    med_iqr = lambda x: f"{np.median(x):.1f} ({np.percentile(x,25):.2f}–{np.percentile(x,75):.2f})"
    
    n_total = len(df)
    n0, n1 = len(group0), len(group1)
    
    res = {
        'Variable': var,
        f'Overall (N = {n_total})': med_iqr(overall),
        f'Non-sepsis group (N = {n0})': med_iqr(group0),
        f'Sepsis group (N = {n1})': med_iqr(group1),
    }
    
    try:
        stat, p = mannwhitneyu(group0, group1, alternative='two-sided')
        res['P-value'] = '<0.001' if p < 0.001 else f'{p:.3f}'
    except ValueError:
        res['P-value'] = 'NA'
    
    results.append(res)
    return results

In [231]:
# Hàm mô tả biến phân loại
def chi_square_test(var, df, results):
    obs = pd.crosstab(df[var], df['sepsis'])
    res = chi2_contingency(obs)
    pval = '<0.001' if res.pvalue < 0.001 else f'{res.pvalue:.3f}'
    
    n_total = len(df)
    n_non = (df['sepsis'] == 0).sum()
    n_sep = (df['sepsis'] == 1).sum()
    
    # Hàng tiêu đề
    results.append({
        'Variable': var,
        f'Overall (N = {n_total})': f'N = {n_total}',
        f'Non-sepsis group (N = {n_non})': '',
        f'Sepsis group (N = {n_sep})': '',
        'P-value': pval
    })
    
    # Chi tiết từng giá trị (Female, Male, v.v.)
    overall_counts = df[var].value_counts()
    overall_perc = (overall_counts / n_total * 100).round(1)
    
    for level in obs.index:
        row = {'Variable': f'  {level}'}
        row[f'Overall (N = {n_total})'] = f"{overall_counts[level]} ({overall_perc[level]}%)"
        row[f'Non-sepsis group (N = {n_non})'] = f"{obs.loc[level,0]} ({obs.loc[level,0]/n_non*100:.1f}%)"
        row[f'Sepsis group (N = {n_sep})'] = f"{obs.loc[level,1]} ({obs.loc[level,1]/n_sep*100:.1f}%)"
        row['P-value'] = ''
        results.append(row)
    return results

In [232]:
# Thực hiện thống kê
results = []

continuous_vars = [
    'age', 'wbc','rdw', 'rbc', 'hemoglobin',
    'platelets', 'aniongap', 'bicarbonate', 'calcium', 'chloride',
    'potassium', 'creatinine', 'glucose', 'triglycerides',
    'alt', 'alp', 'ast', 'pt', 'ptt', 'inr', 'tyg'
]

categorical_vars = ['gender', 'hypertension', 'diabetes', 'aki', 'mortality_28d', 'mortality_90d']

for var in continuous_vars:
    describe_continuous(var, df, results)

for var in categorical_vars:
    chi_square_test(var, df, results)

baseline_table = pd.DataFrame(results)


n_total = len(df)
n_non = (df['sepsis'] == 0).sum()
n_sep = (df['sepsis'] == 1).sum()

cols_final = [
    'Variable',
    f'Overall (N = {n_total})',
    f'Non-sepsis group (N = {n_non})',
    f'Sepsis group (N = {n_sep})',
    'P-value'
]

baseline_table = baseline_table.reindex(columns=cols_final, fill_value='')

baseline_table

,Variable,Overall (N = 656),Non-sepsis group (N = 426),Sepsis group (N = 230),P-value
0,age,55.5 (44.00–67.25),53.0 (42.00–65.00),60.0 (48.00–72.00),<0.001
1,wbc,13.2 (9.57–17.09),11.9 (8.59–15.30),15.8 (12.17–20.84),<0.001
2,rdw,15.3 (14.17–17.10),14.9 (13.87–16.41),16.1 (14.92–18.12),<0.001
3,rbc,3.2 (2.87–3.69),3.3 (2.91–3.78),3.1 (2.79–3.44),<0.001
4,hemoglobin,9.7 (8.74–10.95),10.0 (8.99–11.33),9.2 (8.39–10.38),<0.001
5,platelets,128.0 (71.75–187.25),140.0 (85.00–200.00),107.0 (43.00–170.50),<0.001
6,aniongap,13.8 (12.35–16.31),13.6 (12.21–15.44),14.8 (12.77–17.51),<0.001
7,bicarbonate,23.3 (21.06–25.63),23.8 (21.70–25.82),22.1 (19.77–24.87),<0.001
8,calcium,9.2 (8.80–9.80),9.2 (8.70–9.60),9.4 (8.80–10.47),<0.001
9,chloride,110.0 (106.00–115.00),110.0 (105.00–114.00),112.0 (108.00–116.00),<0.001


# Mối liên hệ gữa TyG và Sepsis

In [233]:

# Chia nhóm TyG theo tứ phân vị
df['TyG_group'] = pd.qcut(df['tyg'], 4, labels=['Q1','Q2','Q3','Q4'])
df['TyG_numeric'] = df['TyG_group'].cat.codes  # cho test p-trend
df.head()

,subject_id,hadm_id,stay_id,intime,age,gender,race,los,diabetes,hypertension,...,los_icu,los_hosp,hosp_mortality,icu_mortality,mortality_7d,mortality_28d,mortality_90d,mortality_1y,TyG_group,TyG_numeric
0,10004606,29242151,30213599,2159-02-20 16:10:03,64,F,White,5.166100,0,1,...,5.166100,14.130556,0,0,0,0,0,0,Q3,2
1,10006441,24120560,37240744,2174-09-12 16:57:04,51,M,White,0.776481,0,0,...,0.776481,15.834722,0,0,0,0,0,0,Q1,0
2,10007795,28477357,31921355,2136-04-22 18:01:13,53,F,White,1.050521,0,1,...,1.050521,21.834722,0,0,0,0,0,0,Q3,2
3,10010888,20162667,33318955,2174-01-09 00:21:00,43,M,White,9.019225,1,0,...,9.019225,17.736806,0,0,0,0,0,0,Q3,2
4,10012206,23961896,37675714,2127-07-04 02:08:00,45,M,White,0.572477,1,1,...,0.572477,10.655556,0,0,0,0,0,0,Q4,3


In [234]:
# Các biến điều chỉnh
covars = ['age', 'gender', 'wbc','rdw', 'rbc', 'hemoglobin','hypertension',
    'platelets', 'aniongap', 'bicarbonate', 'calcium', 'chloride',
    'potassium', 'creatinine', 'alt', 'alp', 'ast', 'pt', 'ptt', 'inr']

In [235]:
# Hàm tính OR và CI
def get_or_ci(model, var):
    coef = model.params[var]
    se = model.bse[var]
    OR = np.exp(coef)
    CI_low = np.exp(coef - 1.96 * se)
    CI_high = np.exp(coef + 1.96 * se)
    return f"{OR:.3f}({CI_low:.3f}–{CI_high:.3f})"

In [236]:
# Các mô hình hồi quy logistic
m1 = smf.logit("sepsis ~ tyg", data=df).fit(disp=False)
m2 = smf.logit("sepsis ~ tyg + age + gender", data=df).fit(disp=False)
m3 = smf.logit("sepsis ~ tyg + " + " + ".join(covars), data=df).fit(disp=False)

# Quartile models (Q1 = ref)
m1_q = smf.logit("sepsis ~ C(TyG_group, Treatment(reference='Q1'))", data=df).fit(disp=False)
m2_q = smf.logit("sepsis ~ C(TyG_group, Treatment(reference='Q1')) + age + gender", data=df).fit(disp=False)
m3_q = smf.logit("sepsis ~ C(TyG_group, Treatment(reference='Q1')) + " + " + ".join(covars), data=df).fit(disp=False)

In [237]:
# Hàm trích xuất OR theo nhóm TyG
def extract_quartiles(model):
    out = {'Q1': 'Ref'}
    for var in model.params.index:
        if "C(TyG_group" in var:
            # var dạng: C(TyG_group, Treatment(reference='Q1'))[T.Q2]
            grp = var.split('.')[-1].replace(']', '').replace('T.', '')
            out[grp] = get_or_ci(model, var)
    # Đảm bảo đủ thứ tự
    for q in ['Q2', 'Q3', 'Q4']:
        if q not in out:
            out[q] = ''
    return out

q1 = extract_quartiles(m1_q)
q2 = extract_quartiles(m2_q)
q3 = extract_quartiles(m3_q)

In [238]:
# P for trend
p1 = smf.logit("sepsis ~ TyG_numeric", data=df).fit(disp=False).pvalues['TyG_numeric']
p2 = smf.logit("sepsis ~ TyG_numeric + age + gender", data=df).fit(disp=False).pvalues['TyG_numeric']
p3 = smf.logit("sepsis ~ TyG_numeric + " + " + ".join(covars), data=df).fit(disp=False).pvalues['TyG_numeric']

# def format_p(p):
#     return "<0.001" if p < 0.001 else f"{p:.3f}"

In [239]:
def format_p(p):
    if p < 0.001:
        return 'p<0.001'
    return f'{p:.3f}'

In [240]:
# Tạo bảng kết quả
table = pd.DataFrame({
    'Model 1': [
        get_or_ci(m1, 'tyg'),
        q1['Q1'], q1['Q2'], q1['Q3'], q1['Q4'],
        format_p(p1)
    ],
    'Model 2': [
        get_or_ci(m2, 'tyg'),
        q2['Q1'], q2['Q2'], q2['Q3'], q2['Q4'],
        format_p(p2)
    ],
    'Model 3': [
        get_or_ci(m3, 'tyg'),
        q3['Q1'], q3['Q2'], q3['Q3'], q3['Q4'],
        format_p(p3)
    ]
}, index=[
    'TyG index', 'Q1', 'Q2', 'Q3', 'Q4', 'P for trend'
])

table

,Model 1,Model 2,Model 3
TyG index,0.616(0.496–0.766),0.670(0.534–0.840),0.740(0.560–0.977)
Q1,Ref,Ref,Ref
Q2,0.572(0.366–0.892),0.576(0.368–0.902),0.482(0.281–0.825)
Q3,0.636(0.409–0.989),0.677(0.433–1.060),0.678(0.399–1.152)
Q4,0.310(0.192–0.501),0.367(0.223–0.605),0.415(0.225–0.765)
P for trend,p<0.001,p<0.001,0.018


In [241]:
df.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'intime', 'age', 'gender', 'race',
       'los', 'diabetes', 'hypertension', 'wbc', 'rbc', 'rdw', 'hemoglobin',
       'platelets', 'aniongap', 'bicarbonate', 'calcium', 'chloride',
       'potassium', 'creatinine', 'glucose', 'triglycerides', 'alt', 'alp',
       'ast', 'pt', 'ptt', 'inr', 'neutrophils_max', 'neutrophils_min',
       'albumin_max', 'albumin_min', 'bun_max', 'bun_min', 'sepsis',
       'sepsis_3', 'npar', 'tyg', 'sofa', 'has_sepsis', 'aki',
       'has_chronic_kidney_disease', 'has_crrt', 'has_mv', 'has_ercp',
       'has_vasopressin', 'los_icu', 'los_hosp', 'hosp_mortality',
       'icu_mortality', 'mortality_7d', 'mortality_28d', 'mortality_90d',
       'mortality_1y', 'TyG_group', 'TyG_numeric'],
      dtype='object')

In [242]:

# # Các biến điều chỉnh
# covars = ['age', 'gender', 'wbc','rdw', 'rbc', 'hemoglobin','hypertension',
#     'platelets', 'aniongap', 'bicarbonate', 'calcium', 'chloride',
#     'potassium', 'creatinine', 'alt', 'alp', 'ast', 'pt', 'ptt', 'inr']

# # Xác định spline cho TyG
# knots = np.percentile(df['tyg'], [5, 27.5, 50, 72.5, 95])
# inner_knots = knots[1:-1]
# spline_formula = f"bs(tyg, knots=({inner_knots[0]}, {inner_knots[1]}, {inner_knots[2]}), degree=3, include_intercept=False)"

# # Fit hai mô hình logistic
# m1 = smf.logit(f"sepsis ~ {spline_formula}", data=df).fit(disp=False)

# formula_adj = f"sepsis ~ {spline_formula} + " + " + ".join(covars)
# m2 = smf.logit(formula_adj, data=df).fit(disp=False)

# # Tạo df dự đoán
# TyG_seq = np.linspace(df['tyg'].min(), df['tyg'].max(), 200)
# df_pred1 = pd.DataFrame({'tyg': TyG_seq})

# df_pred2 = df_pred1.copy()
# for c in covars:
#     if df[c].dtype.kind in 'biufc':
#         df_pred2[c] = df[c].median()
#     else:
#         df_pred2[c] = df[c].mode()[0]

# # Tạo ma trận thiết kế (avoid NameError)
# X1 = dmatrix(m1.model.data.design_info.builder, df_pred1, return_type='dataframe')
# X2 = dmatrix(m2.model.data.design_info.builder, df_pred2, return_type='dataframe')

# # Điểm tham chiếu (median)
# ref_idx = np.argmin(abs(TyG_seq - np.median(df['tyg'])))

# # Hàm bootstrap OR ± 95% CI
# def bootstrap_or(model, X_pred, ref_idx, n_boot=1000, seed=42):
#     np.random.seed(seed)
#     params = model.params.values
#     cov = model.cov_params().values
#     n = X_pred.shape[0]

#     boot_odds = np.zeros((n, n_boot))
#     for i in range(n_boot):
#         sample_params = np.random.multivariate_normal(params, cov)
#         lin_pred = np.dot(X_pred.values, sample_params)
#         prob = 1 / (1 + np.exp(-lin_pred))
#         boot_odds[:, i] = prob / prob[ref_idx]

#     odds = np.median(boot_odds, axis=1)
#     ci_low = np.percentile(boot_odds, 2.5, axis=1)
#     ci_high = np.percentile(boot_odds, 97.5, axis=1)
#     return odds, ci_low, ci_high

# # Tính OR & CI
# odds1, ci1_low, ci1_high = bootstrap_or(m1, X1, ref_idx)
# odds2, ci2_low, ci2_high = bootstrap_or(m2, X2, ref_idx)

# # Kiểm định phi tuyến (Adjusted model)
# m_lin = smf.logit("sepsis ~ tyg", data=df).fit(disp=False)
# LR_nonlin = 2 * (m2.llf - m_lin.llf)
# df_diff = m2.df_model - m_lin.df_model
# p_nonlin = stats.chi2.sf(LR_nonlin, df_diff)

# # Vẽ biểu đồ OR ± 95% CI (giống hình mẫu)
# plt.figure(figsize=(7.5, 6), facecolor="#fffaf2")  # nền sáng dịu

# # Vẽ đường OR và CI ---
# plt.plot(TyG_seq, odds2, color='navy', linewidth=2.5)
# plt.fill_between(TyG_seq, ci2_low, ci2_high, color='skyblue', alpha=0.4)

# # --- Đường tham chiếu ---
# plt.axhline(y=1, color='black', linestyle='--', linewidth=1.2)
# plt.axvline(x=TyG_seq[ref_idx], color='magenta', linestyle='-', linewidth=1.8)

# # --- Thêm các điểm spline (minh họa) ---
# knots_show = np.percentile(df['tyg'], [5, 27.5, 50, 72.5, 95])
# plt.scatter(knots_show, np.interp(knots_show, TyG_seq, odds2),
#             color='blue', s=35, zorder=5)

# # --- Cài đặt trục và nhãn ---
# plt.xlabel("tyg", fontsize=13)
# plt.ylabel("Odds Ratio (95% CI)", fontsize=13)
# plt.title("RCS Prediction Plot", fontsize=15, weight='bold', pad=15)

# # --- Thêm text p-value ---
# plt.text(df["tyg"].min()+0.1,
#          ci2_high.max()*0.95,
#          f"P for Overall <0.001\nP for Nonlinear = {p_nonlin:.3f}",
#          fontsize=12, ha='left', va='top')

# plt.grid(False)
# plt.tight_layout()
# plt.show()



# Phân tích nhóm con 

In [243]:


# Biến số thứ tự để test p-trend
df['TyG_numeric'] = df['TyG_group'].cat.codes + 1


In [244]:
def get_or_ci(model, var):
    coef = model.params[var]
    se   = model.bse[var]
    OR = np.exp(coef)
    CI_low = np.exp(coef - 1.96*se)
    CI_high = np.exp(coef + 1.96*se)
    return f"{OR:.3f}({CI_low:.3f}–{CI_high:.3f})"


In [245]:
df['age_group'] = np.where(df['age'] >= 65, '>=65', '<65')
df['gender_group'] = df['gender']
df['htn_group'] = np.where(df['hypertension'] == 1, 'Yes', 'No')
df['dm_group'] = np.where(df['diabetes'] == 1, 'Yes', 'No')


In [246]:

subgroups = {
    'Age': ('age_group', ['>=65', '<65']),
    'Gender': ('gender_group', ['M', 'F']),
    'Hypertension': ('htn_group', ['Yes', 'No']),
    'Diabetes': ('dm_group', ['Yes', 'No'])
}

def count_case_total(df, var, lvl):
    sub = df[df[var] == lvl]
    case = sub['sepsis'].sum()
    total = sub.shape[0]
    return case, total




In [247]:
def run_subgroup(df, var, lvl, covars):

    sub = df[df[var] == lvl].copy()

    # --- Nếu số ca < 20 thì bỏ qua để tránh lỗi ---
    if sub.shape[0] < 20 or sub['sepsis'].nunique() < 2:
        return {"Q2":"", "Q3":"", "Q4":""}, np.nan
    
    try:
        # Q1 = Ref
        formula = "sepsis ~ C(TyG_group, Treatment(reference='Q1')) + " + " + ".join(covars)
        model = smf.logit(formula, data=sub).fit(disp=False)

        ORs = {}
        for param in model.params.index:
            if "C(TyG_group" in param:
                q = param.split('T.')[1].replace(']','')
                coef = model.params[param]
                se = model.bse[param]
                OR = np.exp(coef)
                L = np.exp(coef - 1.96*se)
                U = np.exp(coef + 1.96*se)
                ORs[q] = f"{OR:.3f}({L:.3f}–{U:.3f})"

        # P-trend
        model_trend = smf.logit("sepsis ~ TyG_numeric + " + " + ".join(covars), data=sub).fit(disp=False)
        p_trend = model_trend.pvalues['TyG_numeric']

        return ORs, p_trend

    except Exception as e:
        # Nếu subgroup lỗi, trả về rỗng
        return {"Q2":"", "Q3":"", "Q4":""}, np.nan


In [248]:
def p_interaction(df, subgroup_var, covars):
    # dùng numeric TyG để kiểm định interaction
    if "TyG_numeric" not in df.columns:
        df["TyG_numeric"] = df["TyG_group"].cat.codes

    formula = "sepsis ~ TyG_numeric * C(" + subgroup_var + ") + " + " + ".join(covars)

    try:
        model = smf.logit(formula, data=df).fit(disp=False)
        return model.pvalues.filter(like="TyG_numeric:C").min()
    except:
        return np.nan


In [249]:
covars_full = ['age','gender','wbc', 'rbc', 'rdw', 'hemoglobin', 'platelets',
           'aniongap', 'bicarbonate', 'calcium', 'chloride', 'potassium', 'creatinine',
           'alt', 'alp', 'ast', 'ptt', 'pt', 'inr']

rows = []

for subgroup_name, (var, levels) in subgroups.items():

    p_int = p_interaction(df, var, covars_full)

    for lvl in levels:

        # CASE & TOTAL
        case, total = count_case_total(df, var, lvl)

        # OR Q2-Q4 + P-trend
        ORs, p_trend = run_subgroup(df, var, lvl, covars_full)

        rows.append({
            "Subgroup": subgroup_name,
            "Level": lvl,
            "CASE": case,
            "TOTAL": total,
            "Q1": "Ref",
            "Q2": ORs.get("Q2",""),
            "Q3": ORs.get("Q3",""),
            "Q4": ORs.get("Q4",""),
            "P for trend": "" if pd.isna(p_trend) else ("<0.001" if p_trend < 0.001 else f"{p_trend:.3f}"),
            "P for interaction": "" if pd.isna(p_int) else ("<0.001" if p_int < 0.001 else f"{p_int:.3f}")
        })

table3 = pd.DataFrame(rows)
table3


,Subgroup,Level,CASE,TOTAL,Q1,Q2,Q3,Q4,P for trend,P for interaction
0,Age,>=65,91,209,Ref,0.449(0.193–1.044),0.561(0.236–1.337),0.209(0.060–0.736),0.020,0.207
1,Age,<65,139,447,Ref,0.616(0.288–1.319),0.892(0.429–1.852),0.609(0.282–1.313),0.380,0.207
2,Gender,M,139,386,Ref,0.383(0.187–0.785),0.501(0.244–1.032),0.323(0.147–0.711),0.012,0.548
3,Gender,F,91,270,Ref,0.735(0.301–1.793),1.136(0.472–2.735),0.765(0.259–2.258),0.916,0.548
4,Hypertension,Yes,76,262,Ref,0.354(0.144–0.870),0.372(0.143–0.971),0.251(0.084–0.752),0.014,0.064
5,Hypertension,No,154,394,Ref,0.464(0.225–0.956),0.846(0.428–1.674),0.535(0.246–1.165),0.304,0.064
6,Diabetes,Yes,74,197,Ref,0.186(0.055–0.627),0.738(0.221–2.468),0.127(0.035–0.463),0.015,0.046
7,Diabetes,No,156,459,Ref,0.621(0.326–1.185),0.725(0.384–1.369),0.758(0.362–1.587),0.489,0.046


## rời rạc -> Chi-square
## liên tục -> nomal = F-test, ko nomal = Kruskal

## Chi-square test
Chọn mức ý nghĩa alpha = 0.05

Giả thuyết

H_0: kiểm tra ảnh hưởng của Male và Female có giống nhau không

H_1: Male và Female khác nhau => (bác bỏ H0 -> pvalue < alpha)


In [250]:
# var = 'gender'
# obs = pd.crosstab(df[var], df['npar_group'])
# res = chi2_contingency(obs)
# res.pvalue


def chi_square_test(var, results):
    obs = pd.crosstab(df[var], df['TyG_group'])
    res = chi2_contingency(obs)
    results.append({
        'Variable': var,
        'Q1': '',
        'Q2': '',
        'Q3': '',
        'Q4': '',
        'pvalue': '<0.001' if res.pvalue < 0.001 else f'{res.pvalue:.3f}'
    })
    percents = obs.div(obs.sum(axis=1), axis=0) * 100
    tabs = obs.astype(str) + '(' + percents.round(1).astype(str) + '%)'
    for name in obs.index:
        dic = {}
        dic['Variable'] = name
        for col in obs.columns:
            dic[col] = tabs.loc[name, col]
        dic['pvalue'] = ''
        results.append(dic)
    return results
results = []
vars = ['gender', 'race','aki', 'diabetes', 'hypertension', 'mortality_28d', 'mortality_90d', 'sepsis']
for var in vars:
    chi_square_test(var, results)
pd.DataFrame(results)



,Variable,Q1,Q2,Q3,Q4,pvalue
0,gender,,,,,0.505
1,F,73(27.0%),70(25.9%),67(24.8%),60(22.2%),
2,M,91(23.6%),94(24.4%),97(25.1%),104(26.9%),
3,race,,,,,0.232
4,Black,24(35.8%),14(20.9%),14(20.9%),15(22.4%),
5,Others,35(18.9%),50(27.0%),50(27.0%),50(27.0%),
6,White,105(26.0%),100(24.8%),100(24.8%),99(24.5%),
7,aki,,,,,0.034
8,0,91(25.0%),76(20.9%),96(26.4%),101(27.7%),
9,1,73(25.0%),88(30.1%),68(23.3%),63(21.6%),


# F-Test

$$H0: \mu Q1 = \mu Q2 = \mu Q3 = \mu Q4 
% $$H0: \mu_{Q_1} = \mu_{Q_2} = \mu_{Q_3} = \mu_{Q_4}$$ 

In [251]:
def f_test(var, results):
    #names = sorted(df['NPAR_group'].unique().tolist())
    names = ['Q1', 'Q2', 'Q3', 'Q4']
    groups = [df[df['TyG_group'] == name][var].dropna() for name in names]
    res = f_oneway(*groups)
    dic = {}
    dic['Variable'] = var
    dic['Total'] = f'{df[var].mean():.1f} ± {df[var].std():.1f}'
    for name, group in zip(names, groups):
        dic[name] = f'{group.mean():.1f} ± {group.std():.1f}'
    dic['pvalue'] = '<0.001' if res.pvalue < 0.001 else f'{res.pvalue:.3f}'
    results.append(dic)
    return results

results = []
vars = ['age',
       'los', 'rbc', 'rdw', 'hemoglobin',
       'platelets', 'aniongap','chloride',
       'potassium', 'creatinine', 'glucose', 'triglycerides', 'alt', 'alp',
       'ast', 'pt', 'ptt', 'inr', 'neutrophils_max', 'neutrophils_min',
       'albumin_max', 'albumin_min', 'bun_max', 'bun_min',
       'npar', 'tyg', 'sofa'
]
for var in vars:
    f_test(var, results)
pd.DataFrame(results)


,Variable,Total,Q1,Q2,Q3,Q4,pvalue
0,age,56.1 ± 16.5,60.8 ± 17.2,59.6 ± 16.0,55.9 ± 15.8,48.1 ± 14.1,<0.001
1,los,9.3 ± 12.8,10.9 ± 17.1,8.5 ± 11.6,10.3 ± 12.3,7.5 ± 8.7,0.057
2,rbc,3.3 ± 0.6,3.3 ± 0.6,3.3 ± 0.6,3.2 ± 0.5,3.4 ± 0.7,0.018
3,rdw,15.9 ± 2.6,16.2 ± 2.3,16.2 ± 2.6,16.2 ± 2.7,15.2 ± 2.5,<0.001
4,hemoglobin,10.0 ± 1.7,9.9 ± 1.6,9.9 ± 1.7,9.8 ± 1.6,10.3 ± 1.8,0.010
5,platelets,142.6 ± 100.6,118.8 ± 78.7,153.3 ± 105.2,145.4 ± 115.9,153.1 ± 95.8,0.005
6,aniongap,14.6 ± 3.5,14.1 ± 3.2,13.9 ± 3.0,14.8 ± 3.3,15.4 ± 4.2,<0.001
7,chloride,110.7 ± 6.8,111.5 ± 6.3,112.0 ± 6.8,110.7 ± 6.5,108.7 ± 7.1,<0.001
8,potassium,4.1 ± 0.4,4.1 ± 0.4,4.1 ± 0.4,4.0 ± 0.4,4.1 ± 0.4,0.052
9,creatinine,1.6 ± 1.5,1.5 ± 1.1,1.5 ± 1.4,1.7 ± 1.7,1.7 ± 1.7,0.464


In [252]:
df.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'intime', 'age', 'gender', 'race',
       'los', 'diabetes', 'hypertension', 'wbc', 'rbc', 'rdw', 'hemoglobin',
       'platelets', 'aniongap', 'bicarbonate', 'calcium', 'chloride',
       'potassium', 'creatinine', 'glucose', 'triglycerides', 'alt', 'alp',
       'ast', 'pt', 'ptt', 'inr', 'neutrophils_max', 'neutrophils_min',
       'albumin_max', 'albumin_min', 'bun_max', 'bun_min', 'sepsis',
       'sepsis_3', 'npar', 'tyg', 'sofa', 'has_sepsis', 'aki',
       'has_chronic_kidney_disease', 'has_crrt', 'has_mv', 'has_ercp',
       'has_vasopressin', 'los_icu', 'los_hosp', 'hosp_mortality',
       'icu_mortality', 'mortality_7d', 'mortality_28d', 'mortality_90d',
       'mortality_1y', 'TyG_group', 'TyG_numeric', 'age_group', 'gender_group',
       'htn_group', 'dm_group'],
      dtype='object')

In [253]:
# def f_test(var, df, results):
#     names = ['Q1', 'Q2', 'Q3', 'Q4']
#     groups = [df[df['TyG_group'] == name][var].dropna() for name in names]
#     res = f_oneway(*groups)
#     dic = {}
#     dic['Variable'] = var
#     dic['Total'] = f"{df[var].mean():.1f} ± {df[var].std():.1f}"
#     for name, group in zip(names, groups):
#         dic[name] = f"{group.mean():.1f} ± {group.std():.1f}"
#     dic['P-value'] = "<0.001" if res.pvalue < 0.001 else f"{res.pvalue:.3f}"
#     results.append(dic)
#     return results
# results = []
# vars = [
#     'age', 'hemoglobin', 'bicarbonate', 'calcium', 'chloride', 'potassium'
# ]
# for var in vars:
#     f_test(var, df, results)
# table = pd.DataFrame(results, columns=['Variable', 'Total', 'Q1', 'Q2', 'Q3', 'Q4', 'P-value'])
# table


# Kruskal-test


In [254]:
def kruskal_test(var, results):
    #names = sorted(df['NPAR_group'].unique().tolist())
    names = ['Q1', 'Q2', 'Q3', 'Q4']
    groups = [df[df['TyG_group'] == name][var].dropna() for name in names]
    res = kruskal(*groups)
    dic = {}
    dic['Variable'] = var
    dic['Total'] = f'{df[var].median():.1f} {df[var].quantile(0.25):.1f} - {df[var].quantile(0.75):.1f}'
    for name, group in zip(names, groups):
        dic[name] = f'{group.median():.1f} {group.quantile(0.25):.1f} - {group.quantile(0.75):.1f}'
    dic['pvalue'] = '<0.001' if res.pvalue < 0.001 else f'{res.pvalue:.3f}'
    results.append(dic)
    return results

results = []
vars = ['age',
       'los', 'rbc', 'rdw', 'hemoglobin',
       'platelets', 'aniongap','chloride',
       'potassium', 'creatinine', 'glucose', 'triglycerides', 'alt', 'alp',
       'ast', 'pt', 'ptt', 'inr', 'neutrophils_max', 'neutrophils_min',
       'albumin_max', 'albumin_min', 'bun_max', 'bun_min',
       'npar', 'tyg', 'sofa'
]
for var in vars:
    kruskal_test(var, results)
pd.DataFrame(results)

,Variable,Total,Q1,Q2,Q3,Q4,pvalue
0,age,55.5 44.0 - 67.2,60.0 49.0 - 75.0,61.0 48.0 - 74.0,56.0 44.8 - 66.2,47.0 37.0 - 58.0,<0.001
1,los,4.1 1.8 - 13.0,4.0 2.0 - 14.2,3.4 1.6 - 13.1,5.5 1.9 - 13.6,3.9 1.6 - 10.0,0.103
2,rbc,3.2 2.9 - 3.7,3.3 2.9 - 3.7,3.2 2.8 - 3.6,3.2 2.9 - 3.5,3.3 2.9 - 3.8,0.046
3,rdw,15.3 14.2 - 17.1,15.7 14.5 - 17.3,15.4 14.3 - 17.5,15.5 14.4 - 17.3,14.6 13.7 - 16.0,<0.001
4,hemoglobin,9.7 8.7 - 10.9,9.6 8.7 - 10.7,9.6 8.5 - 10.8,9.5 8.7 - 10.6,10.0 8.9 - 11.4,0.015
5,platelets,128.0 71.8 - 187.2,110.0 53.5 - 161.0,136.0 84.0 - 200.2,122.0 57.5 - 198.2,140.0 88.0 - 198.2,0.003
6,aniongap,13.8 12.4 - 16.3,13.4 12.0 - 15.6,13.3 11.7 - 15.4,14.4 12.8 - 16.7,14.6 13.0 - 16.9,<0.001
7,chloride,110.0 106.0 - 115.0,111.0 107.0 - 115.0,111.0 107.0 - 116.0,110.0 106.0 - 115.0,109.0 104.0 - 112.0,<0.001
8,potassium,4.0 3.8 - 4.3,4.1 3.9 - 4.3,4.0 3.8 - 4.3,4.0 3.8 - 4.2,4.0 3.8 - 4.3,0.075
9,creatinine,1.0 0.7 - 2.0,1.1 0.7 - 2.1,1.0 0.7 - 1.7,1.1 0.7 - 2.1,0.9 0.6 - 2.0,0.684


In [255]:
df.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'intime', 'age', 'gender', 'race',
       'los', 'diabetes', 'hypertension', 'wbc', 'rbc', 'rdw', 'hemoglobin',
       'platelets', 'aniongap', 'bicarbonate', 'calcium', 'chloride',
       'potassium', 'creatinine', 'glucose', 'triglycerides', 'alt', 'alp',
       'ast', 'pt', 'ptt', 'inr', 'neutrophils_max', 'neutrophils_min',
       'albumin_max', 'albumin_min', 'bun_max', 'bun_min', 'sepsis',
       'sepsis_3', 'npar', 'tyg', 'sofa', 'has_sepsis', 'aki',
       'has_chronic_kidney_disease', 'has_crrt', 'has_mv', 'has_ercp',
       'has_vasopressin', 'los_icu', 'los_hosp', 'hosp_mortality',
       'icu_mortality', 'mortality_7d', 'mortality_28d', 'mortality_90d',
       'mortality_1y', 'TyG_group', 'TyG_numeric', 'age_group', 'gender_group',
       'htn_group', 'dm_group'],
      dtype='object')

# test

In [256]:
df2 = pd.get_dummies(df, columns=['TyG_group'], dtype=int, drop_first=True)
